In [1]:
#= Importy i załadowanie danych =#
using JLD2
using Printf
using Statistics
using LinearAlgebra

include("src/AD.jl")
include("src/NN.jl")

Main.NN

In [2]:
#= Przygotowanie danych =#
file = load("data/imdb_dataset_prepared.jld2")
X_train = Int.(file["X_train"])
y_train = vec(Float32.(file["y_train"]))
X_test  = Int.(file["X_test"])
y_test  = vec(Float32.(file["y_test"]))
embeddings = file["embeddings"]
vocab = file["vocab"]
embedding_dim = size(embeddings, 1)

50

In [3]:
dataset = NN.DataLoader((X_train, y_train), batchsize=64, shuffle=true)

Main.NN.DataLoader((Float32[6391.0 286.0 … 11382.0 8311.0; 143.0 286.0 … 4254.0 2891.0; … ; 12849.0 12849.0 … 12849.0 12849.0; 12849.0 12849.0 … 12849.0 12849.0], Float32[0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]), 64, true, [13897, 38171, 34816, 27382, 13441, 14864, 37966, 36480, 39074, 19440  …  28032, 14720, 28775, 1553, 31040, 30783, 26147, 4929, 5257, 13560])

In [4]:
model = NN.Chain(
    x -> Int.(x),
    NN.Embedding(length(vocab), embedding_dim, embeddings),
    x -> permutedims(x, (2, 1, 3)),
    NN.Conv1D(3, embedding_dim, 8, NN.relu),
    NN.MaxPool1D(8),
    NN.flatten,
    NN.Dense(128, 1, NN.sigmoid)
)

Main.NN.Chain(Any[var"#11#13"(), Main.NN.Embedding(12849, 50, Float32[0.90951 -0.58014 … 0.014323 0.0; -0.20702 -1.1316 … -0.74624 0.0; … ; -0.60396 0.1722 … -0.079873 0.0; -0.31136 1.2622 … -0.89164 0.0], Int64[]), var"#12#14"(), Main.NN.Conv1D(3, 50, 8, Float32[-0.12508114 0.0804346 … 0.11566955 -0.1343976; -0.00033073657 -0.018515617 … 0.057583373 0.036607042; 0.008896333 -0.15128098 … -0.217241 0.17847954;;; -0.13444051 0.12119379 … 0.11521362 0.041223954; -0.0064156493 0.018483639 … -0.018643824 0.09770437; 0.026638625 0.1475353 … -0.102164805 -0.013700087;;; -0.13380253 0.046811525 … 0.009224841 0.008873731; 0.16495852 -0.042695455 … 0.054126672 0.035309345; -0.13027486 0.11699414 … 0.048162248 -0.14051206;;; -0.07384698 0.07186265 … 0.15005428 0.09003914; -0.064339794 0.051264625 … -0.10370785 -0.10909414; 0.014720807 0.02388968 … -0.026621602 0.00426194;;; 0.020512637 -0.02655051 … 0.17569557 -0.058895584; 0.00026864902 -0.008911708 … -0.051849186 0.13235548; 0.0697153 -0.16297

In [5]:
accuracy(m, x, y) = mean((vec(m(x)) .> 0.5) .== (y .> 0.5))

accuracy (generic function with 1 method)

In [6]:
#= Trening modelu =#
opt = NN.Adam(Float32(0.001))
epochs = 5
NN.train_model(model, dataset, X_test, y_test, opt, epochs)

Epoch: 1 (35.59s) 	Train: (l: 0.60, a: 0.68) 	Test: (l: 0.50, a: 0.77) 	Memory allocated: 24760.341 MB
Epoch: 2 (30.06s) 	Train: (l: 0.48, a: 0.78) 	Test: (l: 0.45, a: 0.80) 	Memory allocated: 24373.405 MB
Epoch: 3 (32.49s) 	Train: (l: 0.44, a: 0.80) 	Test: (l: 0.43, a: 0.81) 	Memory allocated: 24381.420 MB
Epoch: 4 (29.66s) 	Train: (l: 0.42, a: 0.81) 	Test: (l: 0.41, a: 0.82) 	Memory allocated: 24385.736 MB
Epoch: 5 (31.98s) 	Train: (l: 0.40, a: 0.83) 	Test: (l: 0.39, a: 0.83) 	Memory allocated: 24388.825 MB
